In [1]:
# code for storing subject and trial info
from psychopy import gui, visual, core, data, event, logging, clock, colors, layout
# GUI for saving data # Store info about the experiment session
expName = 'game'
exType = 'wet'
expInfo = {'participant': 'X03','type': exType, 'expName' : expName, 'sessionNum': 'session1'}
dlg = gui.DlgFromDict(dictionary=expInfo, sortKeys=False, title=expName)
if dlg.OK == False:
    core.quit()  # user pressed cancel

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
c:\Users\Tim de Boer\anaconda3\envs\UnicornBCI\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Kan threadmodus niet wijzigen nadat deze is ingesteld
  warnings.warn(str(err))


In [2]:
import numpy as np
import pandas as pd
import realtime_utils as utils
import torch
import pickle
#INIT
filt_ord = 2
freq_limits = np.asarray([[1,100]]) 
freq_limits_names = ['1_100Hz']
sample_duration = 125
sampling_frequency = 250
electrode_names =  ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
filters = utils.init_filters(freq_limits, sampling_frequency, filt_type = 'bandpass', order=filt_ord)
segments, labels, predictions = [], [], []
total_outlier = 0

c:\Users\Tim de Boer\anaconda3\envs\UnicornBCI\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# init DL model
subject = expInfo['participant']
net = utils.EEGNET()
path = r'final_models/models_for_closedloop/EEGNET_X03_best_finetune_session1' # TODO update path later
net.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
net = net.float()
net.eval()

EEGNET(
  (temporal): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=same, bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (spatial): Sequential(
    (0): Conv2d(8, 16, kernel_size=(8, 1), stride=(1, 1), groups=8, bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=True)
  )
  (seperable): Sequential(
    (0): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=same, groups=16, bias=False)
    (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), padding=same, bias=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=True)
  )
  (avgpool1): AvgPool2d(kernel_size=[1, 5], stride=[1, 5], padding=0)
  (avgpool2): AvgPool2d(kernel_size=[1, 5], stride=[1, 5], padding=0)
  (dropout): Dropout(p=0.25, inplace=False)
  (view): Sequential(
    (0): Flatten()
  )
  (fc2): Li

In [4]:
from pynput.keyboard import Key, Controller
keyboard_game = Controller()

In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#psychopy libraries for running the visual cues
from psychopy import gui, visual, core, data, event, logging, clock, colors, layout
import psychopy.iohub as io
from psychopy.hardware import keyboard
#numpy and pd for data storing and manipulation
import numpy as np
import pandas as pd
from numpy.random import  shuffle
# misc libraries to structure the cues properly and save it with date time and stuff
import time
from datetime import date
from pathlib import Path
import random
import os
import time
import msvcrt
import datetime
# lab streaming layer library to capture the data sent by unicorn EEG headset
from pylsl import StreamInlet, resolve_stream

#change path of folders according to your needs
# Data file name stem = absolute path + name; later add .psyexp, .csv, .log, etc
result_path = Path(f'./Expdata/Subjects/'+exType+'/'+expInfo['participant']+'/'+expInfo['sessionNum']+'/'+expName+'/')
result_path.mkdir(exist_ok=True, parents=True)

columns=['Time','FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8','AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3',
                                  'Battery','Counter','Validation']

data_dict = dict((k, []) for k in columns)
current_seg = pd.DataFrame()
total_MI_outliers = 0
all_MI_segments, all_MI_labels, predictions = [], [], []

MI_dict = {'MI_segments' : [], 'predictions': []}

# below code is for initializing the streaming layer which will help us capture data later
finished = False
streams = resolve_stream()
inlet = StreamInlet(streams[0])

# Auto updating trial numbers
trial_list = []
for instance in os.scandir(result_path):
        if instance.path.endswith('.csv'):
            length = len(instance.path)
            trial_list.append(int(instance.path[length-5]))

if len(trial_list) == 0:
    session = '01'
elif len(trial_list) < 9 :
    session = len(trial_list) + 1
    session = '0' + str(session)
else :
    session = str(len(trial_list) + 1)

print(f"Conducting {expName} experiment for subject :", expInfo['participant'])
print('No. of Practice Trials before :', 2)
print("Trial Number :", session)

print('Actual Trial')
print('Total number of trials as of now :', int(session) + 2)
results_fname = expInfo['participant']+'_'+str(date.today())+'_'+expName+'_'+ expInfo['type']+'_'+session+'.csv'
print("Saving file as .. ", results_fname)


Fs = 250 # sampling frequency of Unicorn EEG cap
temp = []
times = []
#start = time.time()
flush = [17*Fs, 37*Fs, 57*Fs, 77*Fs, 97*Fs, 117*Fs]

initial = 0
final = 125
prediction = -1
outliers = []
key = False
while not finished:

    sample, timestamp = inlet.pull_sample()
    
    res = [timestamp] + sample 
    data_dict = utils.update_data(data_dict,res)

    if len(data_dict['FZ']) % 125 == 0:
        df, initial, final = utils.segment_dict(initial, final, sample_duration, data_dict)
        segment_filt, out, filters = utils.pre_processing(df, electrode_names, filters, 
                        sample_duration, freq_limits_names, sampling_frequency)
        current_seg = utils.concatdata(current_seg,segment_filt)   
        outliers.append(out)   

        # do prediction with current segment and update number
        if current_seg.shape[1] == 500:
            if sum(outliers) > 0:
                total_MI_outliers +=1
                print('OUTLIER')
                if key:
                    keyboard_game.release(key) 
            else:
                all_MI_segments.append(current_seg)
                prediction = utils.do_prediction(current_seg, net)
                predictions.append(int(prediction[0]))
                print(f"prediction: {prediction}") 
                if key:
                    keyboard_game.release(key)
                if prediction[0] == 1:
                    key = Key.right 
                    keyboard_game.press(key)
                elif prediction[0] == 2:
                    key = Key.left
                    keyboard_game.press(key)
            outliers = outliers[1:]   

Conducting game experiment for subject : X01
No. of Practice Trials before : 2
Trial Number : 04
Actual Trial
Total number of trials as of now : 6
Saving file as ..  X01_2022-06-10_game_wet_04.csv
prediction: tensor([0])
prediction: tensor([0])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([0])
prediction: tensor([0])
prediction: tensor([0])
prediction: tensor([0])
prediction: tensor([2])
prediction: tensor([1])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([2])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([2])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([1])
prediction: tensor([1])
prediction:

KeyboardInterrupt: 

In [12]:
# making dictionary into a dataframe for saving it as csv
record_data = pd.DataFrame.from_dict(data_dict)

#saving MI segments in pickle file
MI_dict = {'MI_segments' : [], 'predictions': []}
MI_dict['MI_segments'] = all_MI_segments
MI_dict['predictions'] = predictions

result_path = Path(f"./Expdata/Subjects/{exType}/{subject}/{expInfo['sessionNum']}/{expName}")
exp_type = expInfo['type']
MI_fname = f'{subject}_{str(date.today())}_{expName}_{exp_type}_{session}_MIData.pkl'
print("Saving file MI Data file as .. ", MI_fname)
save_file = open(result_path / MI_fname, "wb")
pickle.dump(MI_dict, save_file)
save_file.close()

#fname = Path('./Expdata/Subjects/'+expInfo['participant']+'/'+ expName + '/'+results_fname)
fname = Path('./Expdata/Subjects/'+exType+'/'+expInfo['participant']+'/'+expInfo['sessionNum']+'/'+expName+'/'+results_fname)
record_data.to_csv(fname, index = False)
print('Trial Ended')

Saving file MI Data file as ..  X01_2022-06-10_game_wet_04_MIData.pkl
Trial Ended
